In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

bounds = [(0, 1200), (0, 1200), (-0.55, 0.55), (-0.55, 0.55)]
x_opt = [679.945148297028709, 1026.06697600004691, 0.118876369094410433, -0.39623348521517826]
y_opt = 5126.4967140071


def G05(X):
    y = 3 * X[0] + 0.000001 * (X[0] ** 3) + 2 * X[1] + (0.000002 / 3) * (X[1] ** 3)

    g1 = -X[3] + X[2] - 0.55
    g2 = -X[2] + X[3] - 0.55
    h3 = 1000 * math.sin(-X[2] - 0.25) + 1000 * math.sin(-X[3] - 0.25) + 894.8 - X[0] - 0.0001
    h4 = 1000 * math.sin(X[2] - 0.25) + 1000 * math.sin(X[2] - X[3] - 0.25) + 894.8 - X[1] - 0.0001
    h5 = 1000 * math.sin(X[3] - 0.25) + 1000 * math.sin(X[3] - X[2] - 0.25) + 1294.8 - 0.0001

    return y, g1, g2, h3, h4, h5

In [4]:
# y = lambda X: G05(X)[0]
# g1 = lambda X: G05(X)[1]
# g2 = lambda X: G05(X)[2]
# h3 = lambda X: G05(X)[3]
# h4 = lambda X: G05(X)[4]
# h5 = lambda X: G05(X)[5]
# x0 = [100,100,0,0]

In [5]:
# maxfunction(h5,x0,bounds)

In [6]:
### 适应度函数
def fitnessG05_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 8880
    fmin = 5126
    epsion = 100
    count = np.sum(np.array(G05(X))[1:6]>0)
    
    return -(G05(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG05_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 8880
    fmin = 5126
    vmax = 5767
    e = 0.1
    n = 5
    
    res = G05(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:6]>0)
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness  


In [7]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-100000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-100000.0)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG05_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            w = 0.85
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG05_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [8]:
PSO = PSO_fit(pN=15000, dim=4, max_iter=1000, bounds=bounds)

In [9]:
PSO.init_Population()

In [ ]:
gbest, gfit = PSO.iterator()

[-25029.09249191 -24568.59924502 -25767.65161573 ...  -8880.
 -25503.65851178 -25556.57623525]
[-24917.82131878 -24568.59924502 -25570.61409004 ...  -7515.01832629
 -25503.65851178  -8442.43395509]
[-24917.82131878 -24568.59924502  -7900.12164779 ...  -7515.01832629
  -8880.          -8442.43395509]
[-7947.35715569 -8733.79702706 -7514.72505207 ... -6829.64336908
 -7916.72142345 -7264.21031075]
[-7828.25614365 -8677.42418034 -7514.72505207 ... -6829.64336908
 -7682.71735297 -7264.21031075]
[-7679.68023433 -7377.65219271 -7210.75486155 ... -6829.64336908
 -7210.75486155 -7264.21031075]
[-7679.68023433 -6894.06638166 -6456.55816325 ... -6829.64336908
 -6708.33434406 -7264.21031075]
[-7287.2057997  -6462.05016114 -5932.65934092 ... -6829.64336908
 -6160.62323031 -7263.4062526 ]
[-7287.2057997  -6313.87764677 -5932.65934092 ... -6364.46402036
 -6160.62323031 -7100.7570896 ]
[-7287.2057997  -6137.94824215 -5932.65934092 ... -6364.46402036
 -6160.62323031 -6784.57446888]
[-7200.52391931 -613

[-5169.54697151 -5169.54697151 -5175.64661872 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5169.54697151 -5169.54697151 -5175.64661872 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5169.54697151 -5169.54697151 -5175.64661872 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5169.54697151 -5169.54697151 -5175.64661872 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5169.54697151 -5169.54697151 -5162.08276036 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5169.54697151 -5169.54697151 -5162.08276036 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5169.54697151 -5169.54697151 -5162.08276036 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5151.4366479  -5169.54697151 -5162.08276036 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5151.4366479  -5169.54697151 -5162.08276036 ... -5176.26926799
 -5147.98127538 -5176.59717441]
[-5151.4366479  -5169.54697151 -5162.08276036 ... -5176.26926799
 -5147.98127538 -5161.12005075]
[-5151.4366479  -5169.54697151

In [ ]:
G05(gbest[0])

In [ ]:
gbest[0]